# SGEMM GPU kernel performance


This data set measures the running time of a matrix-matrix product A*B = C, where all matrices have size 2048 x 2048, using a parameterizable SGEMM GPU kernel with 261400 possible parameter combinations. For each tested combination, 4 runs were performed and their results are reported as the 4 last columns. All times are measured in milliseconds
      
Attribute Information:<br>
Independent variables:
**MWG, NWG:** per-matrix 2D tiling at workgroup level: {16, 32, 64, 128} (integer)<br>
3 **KWG:** inner dimension of 2D tiling at workgroup level: {16, 32} (integer)<br>
4-5 **MDIMC, NDIMC:** local workgroup size: {8, 16, 32} (integer)<br>
6-7 **MDIMA, NDIMB:** local memory shape: {8, 16, 32} (integer)<br>
8 **KWI:** kernel loop unrolling factor: {2, 8} (integer)<br>
9-10 **VWM, VWN:** per-matrix vector widths for loading and storing: {1, 2, 4, 8} (integer)<br>
11-12 **STRM, STRN**: enable stride for accessing off-chip memory within a single thread: {0, 1} (categorical)<br>
13-14 **SA, SB**: per-matrix manual caching of the 2D workgroup tile: {0, 1} (categorical)<br>

Output:<br>
15-18 **Run1, Run2, Run3, Run4:** performance times in milliseconds for 4 independent runs using the same parameters. They range between 13.25 and 3397.08.   

Preprocessing:<br>
Compute the mean of the four Output columns in one column "Run"

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics

import sys
sys.path.insert(1, '../RegressionAlgorithms/')
import linearRegression
from knn import *

### Get the Data

In [ ]:
data = pd.read_csv('sgemm_product.csv', delimiter = ',', engine='python')

In [ ]:
data

### Basic Data Information 

In [ ]:
data.info()

In [ ]:
data.describe()

*Check for missing values*

In [ ]:
print(data.isnull().sum(axis=0))

*Merge the Runs Columns*

In [ ]:
df = pd.DataFrame(data,columns=['Run1 (ms)','Run2 (ms)','Run3 (ms)', 'Run4 (ms)'])
av_row = df.mean(axis=1)

data.drop(columns=['Run1 (ms)','Run2 (ms)','Run3 (ms)', 'Run4 (ms)'])
data['Run'] = av_row

data

### Exploratory Data Analysis

**Run Count**

*Histogram of Run Count Distribution*

In [ ]:
fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
bins = np.arange(0, 3400, 100).tolist()
data['Run'].hist(bins=bins)
plt.xticks(bins)
plt.xlabel('Run Count')

**Run Time vs MWG & NWG**

*Box Plot Run Time vs MWG & NWG*

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=data['MWG'], y=data['Run'])
plt.show()

**Run Time vs KWG**

*Box Plot on Run Time vs KWG*

In [ ]:
plt.figure(figsize=(4, 6))
sns.boxplot(x=data['KWG'], y=data['Run'])
plt.show()

**Run Time vs MDIMC & NDIMC**

*Box Plot on Run Time vs MDIMC & NDIMC*

In [ ]:
plt.figure(figsize=(6, 6))
sns.boxplot(x=data['NDIMC'], y=data['Run'])
plt.show()

**Run Time vs MDIMA & NDIMA**

*Box Plot on Run Time vs MDIMA & NDIMA*

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=data['VWM'], y=data['Run'])
plt.show()

**Run Time vs KWI**

*Box Plot on Run Time vs KWI*

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=data['KWI'], y=data['Run'])
plt.show()

**Run Time vs VWM & VWNN**

*Box Plot on Run Time vs VWM & VWN*

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=data['VWM'], y=data['Run'])
plt.show()

**Run Time vs STRM & STRN**

*Box Plot on Run Time vs STRM & STRN*

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=data['STRM'], y=data['Run'])
plt.show()

**Run Time vs SA & SB**

*Box Plot on Run Time vs SA & SB*

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=data['SA'], y=data['Run'])
plt.show()

## Preprocessing

*We don't have nominal value. No Encoding needed*

**Data preparation**

In [ ]:
X = data.drop('Run', axis=1)
y = data['Run']

*Training and Test Split*

In [ ]:
#split the data in attributes and class as well as training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

## Regression Tasks

*Regression Algorithms from Sklearn*

### Linear Regression

In [ ]:
model = linear_model.LinearRegression().fit(X_train, y_train)

# Make predictions using the testing set
y_pred1 = model.predict(X_test)

# The coefficients
print('Coefficients: \n', model.coef_)
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred1))


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred1))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred1))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred1)))

### KNN Regression

In [ ]:
model = KNeighborsRegressor(n_neighbors=3).fit(X_train, y_train)

# Make predictions using the testing set
y_pred = model.predict(X_test)

# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
data.info()

*Our Regression Algorithms*

### Linear Regression Function

In [ ]:
%load_ext autoreload
%autoreload 2

try:
    del X_train['bias']
except:
    print('no bias to remove X_train')    
try:
    del X_test['bias']
except:
    print('no bias to remove X_test')
try:
    del X['bias']
except:
    print('no bias to remove X')


import linearRegressionNumpy

    
#print('\nMatrix Solution for Comparison:')
#weightsMatrix = linearRegressionNumpy.matrixSolution(X_train, y_train)
#yPredMatrix = linearRegressionNumpy.predictLinearRegression(X_test, weightsMatrix)
#print('weights = ', weightsMatrix)


print('\n Now our own iterative regression algorithm:')    
alpha = [] #[500, 500, 500, 500]
alphaMethod = 'const'
mu = 1
convCritList = [1e5, 1e4, 1e3, 1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
print('epsilon       | sum total error:   | sum relative error:  | iterations | Rsquare   ')
for convergenceCriterion in convCritList:

    weights, score, iterations = linearRegressionNumpy.linearRegression(X_train, y_train, alpha = alpha, mu = mu, 
                                                        convergenceCriterion = convergenceCriterion, lossFunction = 'MSE', 
                                                        alphaMethod = alphaMethod, printOutput = False)
    yPred2 = linearRegressionNumpy.predictLinearRegression(X_test, weights)



    print('{:13E} | {:19}| {:21}| {:11}| {:10}'.format(convergenceCriterion, 
                                        str(np.sum(yPred2-y_pred1)), 
                                        str(np.sum((yPred2-y_pred1)/y_pred1)),
                                        str(iterations),
                                        str(r2_score(y_test, yPred2))))
    #print('sum total error: us iterative vs. scikit = ', np.sum(yPred2-y_pred1))
    #print('sum relative error: us iterative vs. scikit = ', np.sum((yPred2-y_pred1)/y_pred1))

print('yPred = ', yPred2)
print('weights = ', weights)
print('score = ', score)   
    
#print('\n\n\n\n\n')

plt.plot(y_pred1, label = 'scikit')
plt.legend()
plt.show()
plt.plot(yPred2, label = 'us iterative solution')
plt.legend()
plt.show()
plt.plot(yPredMatrix, label = 'us matrix solution')
plt.legend()
plt.show()

plt.plot(yPred2-y_pred1, label = 'total error, us iterative vs. scikit')
plt.legend()
plt.show()
plt.plot(yPredMatrix-y_pred1, label = 'total error, us matrix vs. scikit')
plt.legend()
plt.show()
plt.plot(yPred2-yPredMatrix, label = 'total error, us iterative vs. us matrix')
plt.legend()
plt.show()

plt.plot((yPred2-y_pred1)/y_pred1, label = 'relative error, us iterative vs. scikit')
plt.legend()
plt.show()
plt.plot((yPredMatrix-y_pred1)/y_pred1, label = 'relative error, us matrix vs. scikit')
plt.legend()
plt.show()
plt.plot((yPred2-yPredMatrix)/yPredMatrix, label = 'relative error, us vs. us matrix')
plt.legend()
plt.show()

### KNN

**Dictionary creation to apply the mathematical functions of the algorithm**

In [ ]:
dictionary = data.to_dict('records')

In [ ]:
dictionary

In [ ]:
len(dictionary)

**Forecasting instances**

In [ ]:
y_test

**Algorithm parameters**

In [ ]:
mode = 1 # 1 = KNeighbors; 2 = RadiusNeighbors
n_neighbours = 5
distance_function = 1 # 1 = Euclidean Distance; 2 = Manhattan Distance
radius = 0 # 0 indicates no radius
label = 'Run'
features = ['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)']

**Algorithm initialization**

In [ ]:
knn = KNN(dictionary, label, features, mode, n_neighbours, distance_function, radius)

**Execution of the algorithm (forecasting)**

In [ ]:
results = []

for x in y_test.index:
    #print(x)
    target = dictionary[x-1]
    #print(target)
    result = knn.run(target)
    #print(result)
    results.append(result)

**Predictions**

In [ ]:
predictions = pd.Series(results,index=y_test.index)

In [ ]:
predictions

**Evaluation metrics**

In [ ]:
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, predictions))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))